This is part one of the TCS web scraping project. Here, my goal is to create a Pandas dataframe that includes the links of select titles. After some minimal processing of the scraped data, I save it to a CSV for further web scraping and processing in part two of the TCS project. 

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import datetime as dt
import time
import re

In [2]:
def scrapeTitles(title):
    #Parameters for the Requests Post Form submit
    date = dt.datetime.now().date().strftime("%m/%d/%Y")
    url = "https://tcs.ucop.edu/tcs/jsp/nonAcademicTitlesSearch.htm"
    params = {'titleCode': '', 'titleName': '', 
              'effectiveDate': f'{date}', 
              'titleUnitCode': f'{title}', 
              'occupationalSubgroupCategory': 'All', 
              'linkage': '', 'campus': 'All', 
              '_perPgmPssCheckBox': 'no', 
              'perPgmPssCheckBox': 'on', 
              '_perPgmSmgCheckBox': 'no', 
              'perPgmSmgCheckBox': 'on', 
              '_perPgmMspCheckbox': 'no', 
              'perPgmMspCheckbox': 'on', 
              '_payRepCovCheckBox': 'no', 
              'payRepCovCheckBox': 'on', 
              '_payRepUncCheckBox': 'no', 
              'payRepUncCheckBox': 'on', 
              '_salaryStepCheckBox': 'no', 
              'salaryStepCheckBox': 'on', 
              '_salaryMeritCheckBox': 'no', 
              'salaryMeritCheckBox': 'on', 
              'submitNAT': 'Search'}
   
    #Pass the parameters in a post request to site
    r = requests.post(url, params)
    
    
    #Get the html and parse it into a 
    soup = BeautifulSoup(r.text, 'html.parser')
    
    #Get the main table with the titles
    table = soup.find('table', {'class':'tcs'})
    
    #Create an array of headers:
    headers = []
    for i in soup.find('thead').select('tr:nth-of-type(2) th'):
        headers.append(i.text) 

    #Create values for the dataframe:
    tableBody = soup.find('tbody')


    rows = []
    for row in tableBody.find_all('tr'):
        rows.append(row)

    columns = []  
    for row in rows:
        contents = []
        for d in row.find_all('td'):
            if d.find('a'):
                href = d.find('a')['href']
                contents.append(href)
            else:
                contents.append(d.text.replace('\n', '').strip())
        columns.append(contents)

    #Prep and return dataframe with data and headers
    titlesByCampus = pd.DataFrame(data=columns)
    titlesByCampus.columns = headers
    
    return titlesByCampus

In [3]:
HX = scrapeTitles('HX')
time.sleep(30)
RX = scrapeTitles('RX')
time.sleep(30)
TX = scrapeTitles('TX')
time.sleep(30)

In [4]:
# Function to process scrapped unit data. (ex: creating title stems )

def processScraped(data_arr):
    
    # create empty dataframe
    df = pd.DataFrame()
    
    #append to dataframe, df:
    for i in data_arr:
        df = df.append(i, ignore_index=True)
    
    #Drop unneeded columns:
    df.drop(['Per Pgm', 'OSC', 'Linkage', 'UCPath'], axis=1, inplace=True)
    
    #Prepend the Title URL with stem of URL:
    url_pre="https://tcs.ucop.edu/tcs/jsp/"
    df['Title']= [url_pre+i for i in df['Title']]
    
    #Stem titles to remove levels and provide base name:
    #1. Replace Ast with Assistant where preceding value is Language or Physcn:
    df['Title Name'] = \
    df['Title Name'].replace(r'(?<=LANGUAGE\s)AST|(?<=PHYSCN\s)AST', 'ASSISTANT', regex=True)
    
    #2. Create new column that provides stems of each Title Name:
    patt = r'\s?\d?(\s\w{,2})?\s\w{,2}\s?(N?EX|PD|UCOP)?$|^STF\s|(?<=CLIN LAB SCI)\sSPEC|\sAPPR|\sTRAINEE|\sPRN|(?<!STAGE)\sHELPER|\sAST(\sPD)?$|\sASC(\sAST|\sSR)?$'
    
    df['Primary Title'] = df['Title Name'].str.replace(patt, '', regex=True)
    
    #3. Put Title Grade into it's own column:
    df['Title Grade'] = [y.replace(x, '').strip() for x, y in zip(df['Primary Title'], df['Title Name'])]
    #Remove STF
    
    #4. Create a column for the titleCodes:
    df['Title Code'] = df['Title'].str.extract(r'titleCode=(\d+)&')
    
    #5. Create a column for the campusCodes:
    df['Campus Code'] = df['Title'].str.extract(r'campus=(\d+)$')
    
    #6. Create a column that creates a key for each title from a combo of its titleCode and campusCode:
    df['Title Key'] = df['Title Code']+df['Campus Code']
    
    #4. Reorder columns:
    df = df[['Title Key', 'Title Code', 'Campus Code', 'Business Unit', 'Primary Title', 'Title Grade', 'Title Name', 'TUC', 'Title']]
     
    
    return df

In [6]:
data = processScraped([HX, TX, RX])
data.head(10)

,Title Key,Title Code,Campus Code,Business Unit,Primary Title,Title Grade,Title Name,TUC,Title
0,00542130,005421,30,DVCMP,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
1,00542131,005421,31,DVMED,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
2,00542190,005421,90,IRCMP,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
3,00542191,005421,91,IRMED,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
4,00542140,005421,40,LACMP,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
5,00542141,005421,41,LAMED,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
6,00542160,005421,60,SDCMP,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
7,00542161,005421,61,SDMED,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
8,00542120,005421,20,SFCMP,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...
9,00542121,005421,21,SFMED,DIETITIAN,PD,DIETITIAN PD,HX,https://tcs.ucop.edu/tcs/jsp/nonAcademicTitles...


In [9]:
data.to_csv('UPTETitlesLinks.csv')